In [56]:
import gc
gc.collect()
import torch 
torch.cuda.empty_cache()

In [57]:
import warnings
warnings.simplefilter("ignore", UserWarning)

from __future__ import print_function
from pytorch_lightning.metrics.functional import accuracy

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import pickle
import argparse
from argparse import RawTextHelpFormatter
import fnmatch
import os
import json
import random
import numpy as np
import shutil
import traceback
import copy

import os
from os import listdir
from os.path import isfile, join

import numpy as np

from tqdm import tqdm
import random 
from matplotlib import pyplot as plt

import pandas as pd

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import transforms


import torch
import pytorch_lightning as pl
import os

from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import datasets, transforms

from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint

from torch.nn import functional as F
import torchmetrics

from pytorch_lightning.loggers import TensorBoardLogger
from pl_bolts.callbacks import PrintTableMetricsCallback


import math
import torchvision.models as models
from PIL import Image

import timm

In [99]:
class pre_efficientnet_b6_ap(pl.LightningModule):
    def __init__(self, learning_rate, batch_size):
        
        super(pre_efficientnet_b6_ap, self).__init__()
        
        self.lr = learning_rate
        self.batch_size = batch_size
        self.save_hyperparameters()


        backbone = timm.create_model('tf_efficientnet_b6_ap', pretrained=True)
        layers = list(backbone.children())
        num_filters =layers[-1].in_features
        
        layer1 = layers[0]
        layer2 = layers[1:-5]
        layer3 = layers[-5:-1]
        
        self.seq1 = nn.Sequential(layer1)
        self.seq2 = nn.Sequential(*layer2)
        self.seq3 = nn.Sequential(*layer3)

        num_target_classes = 800
        self.classifier = nn.Linear(num_filters, num_target_classes)

    def forward(self, x):
        
        x = self.seq1(x)
        self.seq2.eval()

        with torch.no_grad():
            
            x = self.seq2(x)
            
        x = self.seq3(x).flatten(1)   
        x = self.classifier(x)


        return F.log_softmax(x, dim=1)

In [102]:
class pre_efficientnet_b6_ap_v2(pl.LightningModule):
    def __init__(self, learning_rate, batch_size):
        
        super(pre_efficientnet_b6_ap_v2, self).__init__()
        
        self.lr = learning_rate
        self.batch_size = batch_size
        self.save_hyperparameters()


        layers = list(pre_efficientnet_b6_ap.load_from_checkpoint('efficientnet_b6_ap_epoch=23-val_loss=8.2352.ckpt').children())
        
        layer1 = [layers[0]] + [layers[1][:2]] + [layers[1][2][0][0]]
        layer2 = [layers[1][2][0][1:]] + [layers[1][2][1:-1]] + [layers[1][2][-1][:-1]]
        layer3 = [layers[1][2][-1][-1]] + [layers[2]]
        
        self.seq1 = nn.Sequential(*layer1)
        self.seq2 = nn.Sequential(*layer2)
        self.seq3 = nn.Sequential(*layer3)

        self.classifier = layers[-1]

    def forward(self, x):
        
        
        self.seq1.eval()
        self.seq2.eval()

        with torch.no_grad():
            
            x = self.seq1(x)
            x = self.seq2(x)
            
        x = self.seq3(x).flatten(1)   
        x = self.classifier(x)


        return F.log_softmax(x, dim=1)

    def configure_optimizers(self):
        
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        lr_scheduler = {'scheduler': torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95),
                        'name': 'expo_lr'}
        return [optimizer], [lr_scheduler]
    
    def training_step(self,batch,batch_idx):
        
        x, y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        
        self.log('train_loss',loss, prog_bar=True , on_step=True, on_epoch=True)
        self.log('train_acc', acc, prog_bar=True , on_step=False, on_epoch=True)
        return loss


    def validation_step(self,batch,batch_idx):
        
        x, y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        
        self.log('val_loss', loss, prog_bar=True , on_step=False, on_epoch=True)
        self.log('val_acc', acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss, acc
    
    def train_dataloader(self):

        train_dir = 'C:/Users/bbb50/Desktop/Python/OCR/t_brain_new_gray_augmented/train'
    
        data_transform = transforms.Compose([
                                    transforms.Resize(size = (224, 224)),
                                    transforms.Grayscale(num_output_channels=3),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        ds = ImageFolder(train_dir, transform=data_transform)

        return DataLoader(ds, batch_size=self.batch_size, shuffle=True)
    
    def val_dataloader(self):
        val_dir = 'C:/Users/bbb50/Desktop/Python/OCR/t_brain_new_gray_augmented/val'
        data_transform = transforms.Compose([
                                    transforms.Resize(size = (224, 224)),
                                    transforms.Grayscale(num_output_channels=3),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        ds = ImageFolder(val_dir, transform=data_transform)

        return DataLoader(ds, batch_size=self.batch_size)

In [103]:

model =  pre_efficientnet_b6_ap_v2(0.075, 64)

lr_logger = LearningRateMonitor()

logger = TensorBoardLogger("tb_pre_efficientnet_b6_ap_v2", name="pre_efficientnet_b6_ap_v2")

early_stopping = EarlyStopping('val_loss', mode='min', patience=5)

model_path = 'C:/Users/bbb50/Desktop/Python/OCR/model_path/pre_efficientnet_b6_ap_v2'
checkpoint_callback = ModelCheckpoint(dirpath=model_path, filename= 'efficientnet_b6_ap_v2_{epoch}-{val_loss:.4f}',
                                      monitor='val_loss', mode='min', save_top_k=5)

trainer = pl.Trainer( gpus=1, limit_train_batches=0.25 ,max_epochs=100, callbacks=[lr_logger, early_stopping, checkpoint_callback], logger=logger,
       ) #gpus=1

# ,limit_train_batches=0.25, limit_val_batches=0.01

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [104]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | seq1       | Sequential | 5.7 K 
1 | seq2       | Sequential | 34.4 M
2 | seq3       | Sequential | 6.4 M 
3 | classifier | Linear     | 1.8 M 
------------------------------------------
42.6 M    Trainable params
0         Non-trainable params
42.6 M    Total params
170.319   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]